In [ ]:
%reset -f
# Pyomo stuff
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# math stuff
import sympy as sym
import numpy as np 

#for animations 
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

import pickle as pkl

from mytoolbox import save_data, print_results, init_from_solution

from mytoolbox.TwoLegsRadau import*

import os
import time

# create the model
m = ConcreteModel()

N = 100
L = 5    # body, upper leg, lower leg
P = 2
DOFs = ['x','z','th']
m.DOF = Set(initialize = DOFs) # for the ground contace positions
m.N = RangeSet(N)
m.L = RangeSet(L)
m.P = RangeSet(P)

to_degree = 180/np.pi
to_rad = np.pi/180

## Parameters

In [ ]:
m.g = Param(initialize = 9.81)

m.len = Param(m.L, initialize = get_len)

m.motor_mass = Param(initialize = 0.487)

m.m = Param(m.L, initialize = get_m)

m.Iy = Param(m.L, initialize = get_I)

# paramters of the aircraft
m.S = Param(initialize = 0.6975) #0.5017)
m.b = Param(initialize =  1.918) #1.73)
m.c = Param(initialize =  0.363) #0.2993)
m.A = Param(initialize = 5.28) #5.9655)
m.e = Param(initialize = 0.858) #0.85)

mbody = sum(m.m[l] for l in range(1,L+1))
BW = mbody*m.g.value

# aerodynamic stuff
m.CL_0 = Param(initialize = 0.243200)  #0.0)
m.CD_0 = Param(initialize = 0.12) #0.07)
m.Cm_0 = Param(initialize = -0.026700) #0.0)

m.CL_a = Param(initialize = 4.240906) #5.1309)
m.Cm_a = Param(initialize = -0.780993) #-0.2954)

m.CL_Q = Param(initialize = 7.046092) #7.7330)
m.Cm_Q = Param(initialize = -7.220962) #-10.2807)

m.CL_de = Param(initialize = 0.419064) #0.71266)
m.Cm_de = Param(initialize = -0.922107) #-1.5853)

m.CL_df = Param(initialize = 0.936323)
m.Cm_df = Param(initialize = 0.111822)

m.rho = Param(initialize =1.225) # air dencity"""
#m.τ = Param(initalize =0.2) # leave this for now.

In [ ]:
m.m.pprint()

## Variables

In [ ]:
m.X = Var(m.N, m.L, m.DOF, m.P)
m.dX = Var(m.N, m.L, m.DOF, m.P)
m.ddX = Var(m.N, m.L, m.DOF, m.P)

m.T = Var(m.N, bounds = (0.0, 40))

# for scaling of angles
to_degree = 180/np.pi
to_rad = np.pi/180

m.de = Var(m.N, bounds = (-25, 25)) # elevator deflection
m.df = Var(m.N, bounds = (-10, 10)) # flap deflection

signs = ['ps', 'ng']
m.sgn = Set(initialize = signs)

m.alpha = Var(m.N, m.P, bounds =(-20, 94)) 
m.V = Var(m.N, m.P, bounds = (0.0, None))

## Constraints

In [ ]:
m.hm = 0.02
m.h = Var(m.N, bounds = (0.8,1.2))

m.integrate_p = Constraint(m.N, m.L, m.DOF, rule = BEuler_p)
    
m.integrate_v = Constraint(m.N, m.L, m.DOF, rule = BEuler_v)

# stop the tail from going through the ground
m.tail_z = Var(m.N, m.P, bounds = (None, 0.0))

m.def_tail_z = Constraint(m.N, m.P, rule = def_tail_z)

m.calc_v = Constraint(m.N, m.P, rule = calc_V)

m.calc_a = Constraint(m.N, m.P, rule = calc_a)

# Radau Stuff

# aux variable to assist with scaling
m.acc = Var(m.N,m.L, m.DOF, m.P)
m.vel = Var(m.N,m.L, m.DOF, m.P)

# RADAU

m.get_acc = Constraint(m.N,m.L, m.DOF, m.P, rule = get_acc)

m.get_vel = Constraint(m.N,m.L, m.DOF, m.P, rule = get_vel)

m.integrate_p3 = Constraint(m.N,m.L, m.DOF, m.P, rule = Radau_p)

m.integrate_v3 = Constraint(m.N,m.L, m.DOF, m.P, rule = Radau_v)

### Joint Connections
1. Hip - $(x_1,z_2)$ and top of link one
2. knee - bottom of link one and top of link 2

In [ ]:
# top of the links
m.top = Var(m.N,m.L,m.DOF,m.P)


m.def_top = Constraint(m.N, m.L, m.DOF,m.P, rule = top_links)

# bottom of the links
m.bottom = Var(m.N,m.L,m.DOF,m.P)


m.def_bottom = Constraint(m.N, m.L, m.DOF, m.P, rule = bottom_links)

for n in range(1,N+1):
    for p in range(1,P+1):
        # set to be the upper bounds because down is positive thus the max vaule for the bottom is zero
        m.bottom[n,3,'z',p].setub(0.0)
        m.bottom[n,2,'z',p].setub(0.0)

        m.bottom[n,5,'z',p].setub(0.0)
        m.bottom[n,4,'z',p].setub(0.0)

# connectons


m.connect = Constraint(m.N,m.L, m.DOF,m.P, rule = connect)

# connetion forces and torques
m.Fj = Var(m.N, m.L, m.DOF, m.P)

# joint angles in the joint space
m.th_joint = Var(m.N, m.L,m.P)

m.def_th_joint = Constraint(m.N,m.L,m.P, rule = def_th_joint)

# angluar velocities in the joint space
m.dth_joint = Var(m.N, m.L,m.P)

m.def_dth_joint = Constraint(m.N, m.L, m.P ,rule = def_dth_joint)

# max conditions for the motor
m.tauMax = Param(initialize = 18) # Nm
m.omegaMax = Param(initialize = 285*180/30) # degrees/s

# control torque
m.Tc = Var(m.N, m.L, bounds =(-m.tauMax/BW, m.tauMax/BW))

# realistic torque constraint.

m.min_torque = Constraint(m.N,m.L, rule = min_torque)

m.max_torque = Constraint(m.N, m.L, rule = max_torque)

# set the bounds on the angles of the joints
for n in range(1,N+1):
    for p in range(1,P+1):
        m.th_joint[n,2,p].setub(0.0)
        m.th_joint[n,2,p].setlb(-90)

        m.th_joint[n,3,p].setub(180)
        m.th_joint[n,3,p].setlb(0.0)

        m.th_joint[n,4,p].setub(0.0)
        m.th_joint[n,4,p].setlb(-90)

        m.th_joint[n,5,p].setub(180)
        m.th_joint[n,5,p].setlb(0.0)

## Floor Contacts

In [ ]:
m.mu = Param(initialize = 0.7)


m.wheel_leg = Param(m.DOF, initialize = get_wheel_lengths)

signs = ['ps','ng']
m.sgns = Set(initialize = signs)

contacts = ['foot1','foot2','wheel']
m.contacts = Set(initialize = contacts)

m.contact_p = Var(m.N, m.contacts,m.P, bounds = (0.0, None))
m.def_contact_p = Constraint(m.N, m.contacts, m.P , rule = def_contact_p)

m.contact_v = Var(m.N, m.contacts, m.sgns, m.P, bounds = (0.0,None))

m.def_contact_v = Constraint(m.N, m.contacts, m.P, rule = def_contact_v) 

m.GRF = Var(m.N, m.contacts, m.DOF, m.sgns, m.P, bounds = (0.0, 5.0))

ground_constraints = ['contact', 'friction', 'slip_ps', 'slip_ng']

m.ground_constraints = Set(initialize = ground_constraints)
m.ground_penalties = Var(m.N, m.contacts, m.ground_constraints, bounds = (0.0,None))

m.friction_cone = Var(m.N, m.contacts, m.P, bounds = (0.0, None))
m.def_friction_cone = Constraint(m.N, m.contacts, m.P, rule = def_friction_cone)

# define the frcicion cone ... triangle

# ground interactions

# m.ground_varA = Var(m.N,m.contacts,m.ground_constraints)#, bounds = (0.0, None))
# m.def_ground_varA = Constraint(m.N,m.contacts,m.ground_constraints, rule = def_ground_varA)
# m.ground_varB = Var(m.N,m.contacts,m.ground_constraints)#, bounds = (0.0, None))
# m.def_ground_varB = Constraint(m.N,m.contacts,m.ground_constraints, rule = def_ground_varB)
# m.ground_comp = Constraint(m.N,m.contacts,m.ground_constraints,rule = ground_comp)

m.contact = Constraint(m.N, m.contacts, rule = contact)

#friction -- ensure that the firictional force only acts when the wheel is stationary

m.friction = Constraint(m.N, m.contacts, rule = friction)
    
#get friction to act in the right direction when slipping

m.slip_ps = Constraint(m.N, m.contacts, rule = slip_ps)

m.slip_ng = Constraint(m.N, m.contacts, rule = slip_ng)


# #_______________________________________________________________________________________________________________
# bound forces at the last node
for dof in DOFs:
    for contact in contacts:
        for sgn in signs:
            m.GRF[N,contact,dof,sgn,P].fix(0.0)
            m.GRF[1,contact,dof,sgn,P].fix(0.0)

## Equations of motion

In [ ]:
# symbolic variables
x1,z1,th1 = sym.symbols(['x1','z1','th1'])
dx1b,dz1b,dth1 = sym.symbols(['dx1b','dz1b','dth1b'])
ddx1b,ddz1b,ddth1 = sym.symbols(['ddx1b','ddz1b','ddth1b'])


de, df, T, V, a = sym.symbols(['de','df','T','V','a'])
m1,I1, = sym.symbols(['m1','I1'])

S, b, c, A, e = sym.symbols(['S','b','c','A','e'])

CL_0,CL_a,CL_Q,CL_de, CL_df = sym.symbols(['C_L_0','C_L_alpha','C_L_Q','C_L_delta_E', 'CL_df'])

CD_0 = sym.symbols('C_D_0')

Cm_0,Cm_a,Cm_Q,Cm_de, Cm_df = sym.symbols(['C_m_0','C_m_alpha','C_m_Q','C_m_delta_E','Cm_df'])

rho = sym.symbols('rho')

g = sym.symbols('g')

# equations of motion
infile = open('Stall_EOM','rb')
data = pkl.load(infile)
infile.close()

EOM1 = data["EOM1"]
EOM2 = data["EOM2"]
EOM3 = data["EOM3"]

# Lambdify the equations of motion

func_map = {'sin':sin,
            'cos':cos
           }

sym_list = [
    x1,z1,th1,
    dx1b, dz1b, dth1,
    ddx1b, ddz1b, ddth1,
    V, T, a,
    de, df,
    m1, I1,
    S, b, c, A, e,
    CD_0,
    CL_0,CL_a,CL_Q,CL_de,CL_df,
    Cm_0,Cm_a,Cm_Q,Cm_de,Cm_df,
    rho, g, sym.pi   
]

lamb_ddx1b = sym.lambdify(sym_list, EOM1, modules = [func_map])
lamb_ddz1b = sym.lambdify(sym_list, EOM2, modules = [func_map])
lamb_ddth1 = sym.lambdify(sym_list, EOM3, modules = [func_map])

# body
def EOM_body(m,n,dof,p):
    if n == 1 and p<P:
        return Constraint.Skip

    var_list = [
        m.X[n,1,'x',p],m.X[n,1,'z',p],m.X[n,1,'th',p]*to_rad,
        m.dX[n,1,'x',p], m.dX[n,1,'z',p], m.dX[n,1,'th',p]*to_rad,
        m.ddX[n,1,'x',p], m.ddX[n,1,'z',p], m.ddX[n,1,'th',p]*to_rad,
        (m.V[n,p]), m.T[n], m.alpha[n,p]*to_rad,
        (m.de[n]*to_rad), (m.df[n]*to_rad),
        m.m[1], m.Iy[1],
        m.S, m.b, m.c, m.A, m.e,
        m.CD_0,
        m.CL_0,m.CL_a,m.CL_Q,m.CL_de,m.CL_df,
        m.Cm_0,m.Cm_a,m.Cm_Q,m.Cm_de,m.Cm_df,
        m.rho, m.g, np.pi   
        ]
    
    # contact forces
    Fcx = (-m.Fj[n,2,'x',p])*cos(m.X[n,1,'th',p]*to_rad) - (-m.Fj[n,2,'z',p])*sin(m.X[n,1,'th',p]*to_rad) # leg 1
    Fcx += (-m.Fj[n,4,'x',p])*cos(m.X[n,1,'th',p]*to_rad) - (-m.Fj[n,4,'z',p])*sin(m.X[n,1,'th',p]*to_rad) # leg 2
    
    Fcz = (-m.Fj[n,2,'x',p])*sin(m.X[n,1,'th',p]*to_rad) + (-m.Fj[n,2,'z',p])*cos(m.X[n,1,'th',p]*to_rad) # leg 1
    Fcz += (-m.Fj[n,4,'x',p])*sin(m.X[n,1,'th',p]*to_rad) + (-m.Fj[n,4,'z',p])*cos(m.X[n,1,'th',p]*to_rad) # leg 2
    
    tauc = -m.Tc[n,2] - m.Tc[n,4]
    
    # ground forces
    Fwx = (m.GRF[n,'wheel','x','ps',p]-m.GRF[n,'wheel','x','ng',p])*cos(m.X[n,1,'th',p]*to_rad)
    Fwx += m.GRF[n,'wheel','z','ng',p]*sin(m.X[n,1,'th',p]*to_rad)
    
    Fwz = (m.GRF[n,'wheel','x','ps',p]-m.GRF[n,'wheel','x','ng',p])*sin(m.X[n,1,'th',p]*to_rad) 
    Fwz += -m.GRF[n, 'wheel','z', 'ng',p]*cos(m.X[n,1,'th',p]*to_rad)
    
    M_front = (Fwx) * m.wheel_leg['z']
    M_front+= (Fwz) * m.wheel_leg['x']
    
    S = BW
    
    if dof == 'x':
        return lamb_ddx1b(*var_list) == S*(Fcx + Fwx)/m.m[1]
    if dof == 'z':
        return lamb_ddz1b(*var_list) == S*(Fcz + Fwz)/m.m[1]
    if dof == 'th':
        return lamb_ddth1(*var_list) == S*(tauc + M_front)/m.Iy[1]
m.EOM_body = Constraint(m.N, m.DOF,m.P, rule = EOM_body)

# leg 1

def EOM_L2(m,n,dof,p):
    if n == 1 and p<P:
        return Constraint.Skip
    
    S = BW
    # contact forces
    Fcx = m.Fj[n,2,'x',p] - m.Fj[n,3,'x',p]
    Fcz = m.Fj[n,2,'z',p] - m.Fj[n,3,'z',p]
    
    Mtop = -m.Fj[n,2,'x',p]*0.5*m.len[2]*cos(m.X[n,2,'th',p]*to_rad) + m.Fj[n,2,'z',p]*0.5*m.len[2]*sin(m.X[n,2,'th',p]*to_rad)
    Mbottom = -m.Fj[n,3,'x',p]*0.5*m.len[2]*cos(m.X[n,2,'th',p]*to_rad) + m.Fj[n,3,'z',p]*0.5*m.len[2]*sin(m.X[n,2,'th',p]*to_rad)
    
    # no ground forces
    
    if dof == 'x':
        return m.ddX[n,2,dof,p] == S*Fcx/m.m[2]
    if dof == 'z':
        return m.ddX[n,2,dof,p] == S*Fcz/m.m[2] + m.g*m.m[2] # yes positive z is down
    if dof == 'th':
        return m.ddX[n,2,dof,p] == S*(Mtop + Mbottom + m.Tc[n,2] - m.Tc[n,3])/m.Iy[2] - m.motor_mass*m.g*0.5*m.len[2]*sin(m.X[n,2,'th',p]*to_rad)/m.Iy[2]
m.EOM_L2 = Constraint(m.N,m.DOF,m.P, rule = EOM_L2)

def EOM_L3(m,n,dof,p):
    if n == 1 and p<P:
        return Constraint.Skip
    
    S = BW
    
    Fcx = m.Fj[n,3,'x',p] + (m.GRF[n,'foot1','x','ps',p]-m.GRF[n,'foot1','x','ng',p])
    Fcz = m.Fj[n,3,'z',p] - m.GRF[n,'foot1','z','ng',p]
    
    Mtop = -m.Fj[n,3,'x',p]*0.5*m.len[3]*cos(m.X[n,3,'th',p]*to_rad) + m.Fj[n,3,'z',p]*0.5*m.len[3]*sin(m.X[n,3,'th',p]*to_rad)    
    Mbottom = (m.GRF[n,'foot1','x','ps',p]-m.GRF[n,'foot1','x','ng',p])*0.5*m.len[3]*cos(m.X[n,3,'th',p]*to_rad) + m.GRF[n,'foot1','z','ng',p]*0.5*m.len[3]*sin(m.X[n,3,'th',p]*to_rad)
    
    if dof == 'x':
        return m.ddX[n,3,dof,p] == S*Fcx/m.m[3]
    if dof == 'z':
        return m.ddX[n,3,dof,p] == S*Fcz/m.m[3] + m.g*m.m[3] # yes positive z is down
    if dof == 'th':
        return m.ddX[n,3,dof,p] == S*(Mtop + Mbottom + m.Tc[n,3])/m.Iy[3]
m.EOM_L3 = Constraint(m.N,m.DOF,m.P, rule = EOM_L3)

# leg 2
def EOM_L4(m,n,dof,p):
    if n == 1 and p<P:
        return Constraint.Skip
    
    S = BW
    # contact forces
    Fcx = m.Fj[n,4,'x',p] - m.Fj[n,5,'x',p]
    Fcz = m.Fj[n,4,'z',p] - m.Fj[n,5,'z',p]
    
    Mtop = -m.Fj[n,4,'x',p]*0.5*m.len[4]*cos(m.X[n,4,'th',p]*to_rad) + m.Fj[n,4,'z',p]*0.5*m.len[4]*sin(m.X[n,4,'th',p]*to_rad)
    Mbottom = -m.Fj[n,5,'x',p]*0.5*m.len[4]*cos(m.X[n,4,'th',p]*to_rad) + m.Fj[n,5,'z',p]*0.5*m.len[4]*sin(m.X[n,4,'th',p]*to_rad)
    
    # no ground forces
    
    if dof == 'x':
        return m.ddX[n,2,dof,p] == S*Fcx/m.m[4]
    if dof == 'z':
        return m.ddX[n,2,dof,p] == S*Fcz/m.m[4] + m.g*m.m[4] # yes positive z is down
    if dof == 'th':
        return m.ddX[n,2,dof,p] == S*(Mtop + Mbottom + m.Tc[n,4] - m.Tc[n,5])/m.Iy[4] - m.motor_mass*m.g*0.5*m.len[4]*sin(m.X[n,4,'th',p]*to_rad)/m.Iy[4]
m.EOM_L4 = Constraint(m.N,m.DOF,m.P, rule = EOM_L4)

def EOM_L5(m,n,dof,p):
    if n == 1 and p<P:
        return Constraint.Skip
    
    S = BW
    
    Fcx = m.Fj[n,5,'x',p] + (m.GRF[n,'foot2','x','ps',p]-m.GRF[n,'foot2','x','ng',p])
    Fcz = m.Fj[n,5,'z',p] - m.GRF[n,'foot2','z','ng',p]
    
    Mtop = -m.Fj[n,5,'x',p]*0.5*m.len[5]*cos(m.X[n,5,'th',p]*to_rad) + m.Fj[n,5,'z',p]*0.5*m.len[5]*sin(m.X[n,5,'th',p]*to_rad)    
    Mbottom = (m.GRF[n,'foot2','x','ps',p]-m.GRF[n,'foot2','x','ng',p])*0.5*m.len[5]*cos(m.X[n,5,'th',p]*to_rad) + m.GRF[n,'foot2','z','ng',p]*0.5*m.len[5]*sin(m.X[n,5,'th',p]*to_rad)
    
    if dof == 'x':
        return m.ddX[n,3,dof,p] == S*Fcx/m.m[5]
    if dof == 'z':
        return m.ddX[n,3,dof,p] == S*Fcz/m.m[5] + m.g*m.m[5] # yes positive z is down
    if dof == 'th':
        return m.ddX[n,3,dof,p] == S*(Mtop + Mbottom + m.Tc[n,5])/m.Iy[5]
m.EOM_L5 = Constraint(m.N,m.DOF,m.P, rule = EOM_L5)

## Cost function

In [ ]:
m.Cost = Objective(rule = minDistance)

## Initialize

In [ ]:
def Euler_mode(m):
    m.integrate_p.activate()
    m.integrate_v.activate()
    
    m.integrate_p3.deactivate()
    m.integrate_v3.deactivate()
    m.get_acc.deactivate()
    m.get_vel.deactivate()
    
    for n in range(2,N+1):
        for p in range(1,P):
            m.def_tail_z[n,p].deactivate()
            m.calc_v[n,p].deactivate()
            m.calc_a[n,p].deactivate()
            
            for dof in DOFs:
                m.EOM_body[n,dof,p].deactivate()
                m.EOM_L2[n,dof,p].deactivate()
                m.EOM_L3[n,dof,p].deactivate()
                m.EOM_L4[n,dof,p].deactivate()
                m.EOM_L5[n,dof,p].deactivate()
                
                for l in range(2,L+1):
                    if dof != 'th':
                        m.def_top[n,l,dof,p].deactivate()
                        m.def_bottom[n,l,dof,p].deactivate()
                        m.connect[n,l,dof,p].deactivate()
                        
                        m.def_th_joint[n,l,p].deactivate()
                        m.def_dth_joint[n,l,p].deactivate()
                        
                        
                    for contact in contacts:
                        m.def_contact_p[n,contact,p].deactivate()
                        m.def_contact_v[n,contact,p].deactivate()
                        m.def_friction_cone[n,contact,p].deactivate()
                        
    for n in range(1,N+1):
        for p in range(1,P):
            m.alpha[n,p].fix(0.0)
            m.V[n,p].fix(0.0)
            m.tail_z[n,p].fix(0.0)
            for l in range(1,L+1):
                if  l > 1:
                    m.th_joint[n,l,p].fix(0.0)
                    m.dth_joint[n,l,p].fix(0.0)
                
                for dof in DOFs:
                    m.X[n,l,dof,p].fix(0.0)
                    m.dX[n,l,dof,p].fix(0.0)
                    m.ddX[n,l,dof,p].fix(0.0)
                    
                    
                    if l > 1 and dof != 'th':
                        m.top[n,l,dof,p].fix(0.0)
                        m.bottom[n,l,dof,p].fix(0.0)
                        m.Fj[n,l,dof,p].fix(0.0)
                        
            for contact in contacts:
                m.contact_p[n,contact,p].fix(0.0)
                for sgn in signs:
                    m.friction_cone[n,contact,p].fix(0.0)
                    m.contact_v[n,contact, sgn, p].fix(0.0)
                    for dof in DOFs:
                        m.GRF[n,contact,dof, sgn, p].fix(0.0)
    return m

def Radau_mode(m):
    m.integrate_p.deactivate()
    m.integrate_v.deactivate()
    
    m.integrate_p3.activate()
    m.integrate_v3.activate()
    m.get_acc.activate()
    m.get_vel.activate()
    
    for n in range(2,N+1):
        for p in range(1,P):
            m.def_tail_z[n,p].activate()
            m.calc_v[n,p].activate()
            m.calc_a[n,p].activate()
            
            for dof in DOFs:
                m.EOM_body[n,dof,p].activate()
                m.EOM_L2[n,dof,p].activate()
                m.EOM_L3[n,dof,p].activate()
                m.EOM_L4[n,dof,p].activate()
                m.EOM_L5[n,dof,p].activate()
                
                for l in range(2,L+1):
                    if dof != 'th':
                        m.def_top[n,l,dof,p].activate()
                        m.def_bottom[n,l,dof,p].deactivate()
                        m.connect[n,l,dof,p].activate()
                        
                        m.def_th_joint[n,l,p].activate()
                        m.def_dth_joint[n,l,p].activate()
                        
                        
                    for contact in contacts:
                        m.def_contact_p[n,contact,p].activate()
                        m.def_contact_v[n,contact,p].activate()
                        m.def_friction_cone[n,contact,p].activate()
                        
    for n in range(2,N+1):
        for p in range(1,P):
            m.alpha[n,p].unfix()
            m.V[n,p].unfix()
            m.tail_z[n,p].unfix()
            for l in range(1,L+1):
                if  l > 1:
                    m.th_joint[n,l,p].unfix()
                    m.dth_joint[n,l,p].unfix()
                
                for dof in DOFs:
                    m.X[n,l,dof,p].unfix()
                    m.dX[n,l,dof,p].unfix()
                    m.ddX[n,l,dof,p].unfix()
                    
                    
                    if l > 1 and dof != 'th':
                        m.top[n,l,dof,p].unfix()
                        m.bottom[n,l,dof,p].unfix()
                        m.Fj[n,l,dof,p].unfix()
                        
            for contact in contacts:
                m.contact_p[n,contact,p].unfix()
                for sgn in signs:
                    m.friction_cone[n,contact,p].unfix()
                    m.contact_v[n,contact, sgn, p].unfix()
                    for dof in DOFs:
                        m.GRF[n,contact,dof, sgn, p].unfix()
    return m



def init_midpoints(m):
    for n in range(2,N+1):
        for p in range(1,P):
            m.alpha[n,p].value = m.alpha[n,P].value
            m.V[n,p].value = m.V[n,P].value
            m.tail_z[n,p].value = m.tail_z[n,P].value
            for l in range(1,L+1):
                if  l > 1:
                    m.th_joint[n,l,p].value = m.th_joint[n,l,P].value
                    m.dth_joint[n,l,p].value = m.dth_joint[n,l,P].value
                
                for dof in DOFs:
                    m.X[n,l,dof,p].value = m.X[n,l,dof,P].value
                    m.dX[n,l,dof,p].value = m.dX[n,l,dof,P].value
                    m.ddX[n,l,dof,p].value = m.ddX[n,l,dof,P].value
                    
                    
                    if l > 1 and dof != 'th':
                        m.top[n,l,dof,p].value = m.top[n,l,dof,P].value
                        m.bottom[n,l,dof,p].value = m.bottom[n,l,dof,P].value
                        m.Fj[n,l,dof,p].value = m.Fj[n,l,dof,P].value
                        
            for contact in contacts:
                m.contact_p[n,contact,p].value = m.contact_p[n,contact,P].value
                for sgn in signs:
                    m.friction_cone[n,contact,p].value = m.friction_cone[n,contact,P].value
                    m.contact_v[n,contact, sgn, p].value = m.contact_v[n,contact, sgn, P].value
                    for dof in DOFs:
                        m.GRF[n,contact,dof, sgn, p].value = m.GRF[n,contact,dof, sgn, P].value
    return m

In [ ]:
# initual guess
# whith wheels the plane lands at around 25m so initual guess will be a linear line from the start position
# to z = 0 x = 25
start_height = -18.0
finish_x = 10 
# get the trim conditions
infile = open('TrimConditions','rb')
data = pkl.load(infile)
infile.close()

alpha_t = data['alpha_t']
delta_e_t = data['delta_e_t']
theta_t = alpha_t 
T_t = data['T_t']
V_t = data['V_t']

guess_x1 = np.linspace(0,25, N)
guess_z1 = guess_x1*(-start_height)/finish_x + start_height

guess_th1 = np.ones(N)*(theta_t - theta_t/finish_x * guess_x1)
guess_de = np.ones(N)*delta_e_t
guess_df = np.ones(N)* np.pi/18
guess_V = np.ones(N)*V_t - guess_x1*18/finish_x
guess_T = np.ones(N)*T_t - guess_x1*T_t/finish_x

guess_th2 = np.ones(round(N*2/3))*np.pi/4
guess_th2 = np.append(guess_th2, np.linspace(np.pi/4, -np.pi/2, round(N/3)))                
guess_th3 = np.ones(N)*np.pi/4

guess_x2 = guess_x1 + 0.5*m.len[2]*np.sin(guess_th2)
guess_x3 = guess_x1 + m.len[2]*np.sin(guess_th2) + 0.5 * m.len[3]*np.sin(guess_th3)

guess_z2 = guess_z1 + 0.5*m.len[2]*np.cos(guess_th2)
guess_z3 = guess_z1 + m.len[2]*np.cos(guess_th2) + 0.5 * m.len[3]*np.cos(guess_th3)

guess_tau2 = - m.m[2]*m.g*0.5*m.len[2]*np.sin(guess_th2)/BW
guess_tau3 = - m.m[3]*m.g*0.5*m.len[3]*np.sin(guess_th3)/BW

for n in range(1,N+1):
    m.alpha[n,P].value = alpha_t*to_degree
    m.de[n].value = guess_de[n-1]*to_degree
    m.df[n].value = guess_df[n-1]*to_degree
    m.V[n,P].value = guess_V[n-1]
    m.T[n].value = guess_T[n-1]
    
    m.X[n,1,'x',P].value = guess_x1[n-1]
    m.X[n,2,'x',P].value = guess_x2[n-1]
    m.X[n,3,'x',P].value = guess_x3[n-1]
    m.X[n,4,'x',P].value = guess_x2[n-1]
    m.X[n,5,'x',P].value = guess_x3[n-1]
    
    m.X[n,1,'z',P].value = guess_z1[n-1]
    m.X[n,2,'z',P].value = guess_z2[n-1]
    m.X[n,3,'z',P].value = guess_z3[n-1]
    m.X[n,4,'z',P].value = guess_z2[n-1]
    m.X[n,5,'z',P].value = guess_z3[n-1]
    
    m.X[n,1,'th',P].value = guess_th1[n-1]*to_degree
    m.X[n,2,'th',P].value = guess_th2[n-1]*to_degree
    m.X[n,3,'th',P].value = guess_th3[n-1]*to_degree
    m.X[n,4,'th',P].value = guess_th2[n-1]*to_degree
    m.X[n,5,'th',P].value = guess_th3[n-1]*to_degree
    
    m.Tc[n,2].value = guess_tau2[n-1]
    m.Tc[n,3].value = guess_tau3[n-1]
    m.Tc[n,4].value = guess_tau2[n-1]
    m.Tc[n,5].value = guess_tau3[n-1]

In [ ]:
# initual conditions

m.calc_a[1,P].deactivate()

# Position
m.X[1,1,'x',P].value = 0.0
m.X[1,1,'z',P].value = start_height
m.X[1,1,'th',P].value = 0.03560531051013316*to_degree

m.X[1,1,'x',P].fixed = True 
m.X[1,1,'z',P].fixed = True 
m.X[1,1,'th',P].fixed = True 

m.X[1,2,'th',P].value = np.pi/4*to_degree
m.X[1,2,'x',P].value = m.X[1,1,'x',P].value + 0.5*m.len[2]*np.sin(m.X[1,2,'th',P].value*to_rad) 
m.X[1,2,'z',P].value = m.X[1,1,'z',P].value + 0.5*m.len[2]*np.cos(m.X[1,2,'th',P].value*to_rad)

m.X[1,3,'th',P].value = np.pi/4*to_degree
m.X[1,3,'x',P].value = m.X[1,1,'x',P].value + m.len[2]*np.sin(m.X[1,2,'th',P].value*to_rad) + 0.5*m.len[3]*np.sin(m.X[1,3,'th',P].value*to_rad) 
m.X[1,3,'z',P].value = m.X[1,1,'z',P].value + m.len[2]*np.cos(m.X[1,2,'th',P].value*to_rad) + 0.5*m.len[3]*np.cos(m.X[1,3,'th',P].value*to_rad)

m.X[1,4,'th',P].value = np.pi/4*to_degree
m.X[1,4,'x',P].value = m.X[1,1,'x',P].value + 0.5*m.len[2]*np.sin(m.X[1,2,'th',P].value*to_rad) 
m.X[1,4,'z',P].value = m.X[1,1,'z',P].value + 0.5*m.len[2]*np.cos(m.X[1,2,'th',P].value*to_rad)

m.X[1,5,'th',P].value = np.pi/4*to_degree
m.X[1,5,'x',P].value = m.X[1,1,'x',P].value + m.len[2]*np.sin(m.X[1,2,'th',P].value*to_rad) + 0.5*m.len[3]*np.sin(m.X[1,3,'th',P].value*to_rad) 
m.X[1,5,'z',P].value = m.X[1,1,'z',P].value + m.len[2]*np.cos(m.X[1,2,'th',P].value*to_rad) + 0.5*m.len[3]*np.cos(m.X[1,3,'th',P].value*to_rad)


# Velocity
m.dX[1,1,'x',P].value = V_t * cos(alpha_t)
m.dX[1,1,'z',P].value = V_t * sin(alpha_t)
m.dX[1,1,'th',P].value = 0.0

m.dX[1,1,'x',P].fixed = True
m.dX[1,1,'z',P].fixed = True
m.dX[1,1,'th',P].fixed = True
#----
m.dX[1,2,'x',P].value = V_t * cos(alpha_t)
m.dX[1,2,'z',P].value = V_t * sin(alpha_t)
m.dX[1,2,'th',P].value = 0.0

m.dX[1,2,'x',P].fixed = True
m.dX[1,2,'z',P].fixed = True
m.dX[1,2,'th',P].fixed = True
#----
m.dX[1,3,'x',P].value = V_t * cos(alpha_t)
m.dX[1,3,'z',P].value = V_t * sin(alpha_t)
m.dX[1,3,'th',P].value = 0.0

m.dX[1,3,'x',P].fixed = True
m.dX[1,3,'z',P].fixed = True
m.dX[1,3,'th',P].fixed = True
#----
m.dX[1,4,'x',P].value = V_t * cos(alpha_t)
m.dX[1,4,'z',P].value = V_t * sin(alpha_t)
m.dX[1,4,'th',P].value = 0.0

m.dX[1,4,'x',P].fixed = True
m.dX[1,4,'z',P].fixed = True
m.dX[1,4,'th',P].fixed = True
#----
m.dX[1,5,'x',P].value = V_t * cos(alpha_t)
m.dX[1,5,'z',P].value = V_t * sin(alpha_t)
m.dX[1,5,'th',P].value = 0.0

m.dX[1,5,'x',P].fixed = True
m.dX[1,5,'z',P].fixed = True
m.dX[1,5,'th',P].fixed = True

# Acceleration
#---
m.ddX[1,1,'x',P].value = 0.0
m.ddX[1,1,'z',P].value = 0.0
m.ddX[1,1,'th',P].value = 0.0

m.ddX[1,1,'x',P].fixed = True
m.ddX[1,1,'z',P].fixed = True
m.ddX[1,1,'th',P].fixed = True
#---
m.ddX[1,2,'x',P].value = 0.0
m.ddX[1,2,'z',P].value = 0.0
m.ddX[1,2,'th',P].value = 0.0

m.ddX[1,2,'x',P].fixed = True
m.ddX[1,2,'z',P].fixed = True
m.ddX[1,2,'th',P].fixed = True
#---
m.ddX[1,3,'x',P].value = 0.0
m.ddX[1,3,'z',P].value = 0.0
m.ddX[1,3,'th',P].value = 0.0

m.ddX[1,3,'x',P].fixed = True
m.ddX[1,3,'z',P].fixed = True
m.ddX[1,3,'th',P].fixed = True
#---
m.ddX[1,4,'x',P].value = 0.0
m.ddX[1,4,'z',P].value = 0.0
m.ddX[1,4,'th',P].value = 0.0

m.ddX[1,4,'x',P].fixed = True
m.ddX[1,4,'z',P].fixed = True
m.ddX[1,4,'th',P].fixed = True

#---
m.ddX[1,5,'x',P].value = 0.0
m.ddX[1,5,'z',P].value = 0.0
m.ddX[1,5,'th',P].value = 0.0

m.ddX[1,5,'x',P].fixed = True
m.ddX[1,5,'z',P].fixed = True
m.ddX[1,5,'th',P].fixed = True


# final conditions
m.dX[N,1,'x',P].value = 0.0 #1e-20
m.dX[N,1,'z',P].value = 0.0
m.dX[N,1,'th',P].value = 0.0

m.dX[N,1,'x',P].fixed = True
m.dX[N,1,'z',P].fixed = True
m.dX[N,1,'th',P].fixed = True

m.dX[N,2,'th',P].value = 0.0
m.dX[N,2,'th',P].fixed = True 

m.dX[N,3,'th',P].value = 0.0
m.dX[N,3,'th',P].fixed = True


m.GRF[N,'foot1','z','ng',P].value = 1/2
m.GRF[N,'foot2','z','ng',P].value = 1/2
m.GRF[N,'wheel','z','ng',P].value = 0


m.contact_p[N,'foot1',P].setub(0.01)
m.contact_p[N,'foot2',P].setub(0.01)

# Solve

In [ ]:
m = Euler_mode(m)
opt = init_opt(1e-6)
start = time.time()
results = opt.solve(m, tee = True)
finish = time.time()

In [ ]:
print_results(m, results, start, finish)

In [ ]:
m = Radau_mode(m)
m = init_midpoints(m)
opt = init_opt()
start = time.time()
results = opt.solve(m, tee = True)
finish = time.time()

In [ ]:
print_results(m, results, start, finish)

In [ ]:
save = input('Save Data [y/n]')
plt.figure(figsize = (15,10))

n_force = np.linspace(1,N,N*P)
n_penalty = np.linspace(1,N,N)

# wheel
normal_p = list(m.ground_penalties[:, 'wheel', 'contact'].value)
friction_p = list(m.ground_penalties[:, 'wheel', 'friction'].value)
slip_ps_p = list(m.ground_penalties[:, 'wheel', 'slip_ps'].value)
slip_ng_p = list(m.ground_penalties[:, 'wheel', 'slip_ng'].value)


normal_f = list(m.GRF[:, 'wheel', 'z', 'ng',:].value)
x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        x_f.append(m.GRF[n, 'wheel', 'x', 'ps',p].value - m.GRF[n, 'wheel', 'x', 'ng',p].value) 

plt.subplot(3,2,1)
plt.step(n_penalty,normal_p,where = 'post', label ="Contact")
plt.step(n_penalty,friction_p,where = 'post', label ="Friction")
plt.step(n_penalty,slip_ps_p,where = 'post', label ="Slip +")
plt.step(n_penalty,slip_ng_p,where = 'post', label ="Slip -")
plt.title("Wheel Penalty")
plt.legend()
plt.grid()
plt.axis([1,N,None,None])

plt.subplot(3,2,2)
plt.step(n_force, normal_f,where = 'post', label = 'z')
plt.step(n_force, x_f,where = 'post', label = 'x')
#plt.plot(z,':', color = 'Grey', label = 'z position of the front wheel')
plt.title("Wheel Forces")
plt.legend()
plt.grid()
plt.axis([1,N,None,None])

# foot 2
normal_p = list(m.ground_penalties[:, 'foot1', 'contact'].value)
friction_p = list(m.ground_penalties[:, 'foot1', 'friction'].value)
slip_ps_p = list(m.ground_penalties[:, 'foot1', 'slip_ps'].value)
slip_ng_p = list(m.ground_penalties[:, 'foot1', 'slip_ng'].value)

normal_f = list(m.GRF[:, 'foot1', 'z', 'ng',:].value)
x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        x_f.append(m.GRF[n, 'foot1', 'x', 'ps',p].value - m.GRF[n, 'foot1', 'x', 'ng',p].value) 


plt.subplot(3,2,3)
plt.step(n_penalty, normal_p,where = 'post', label ="Contact")
plt.step(n_penalty, friction_p,where = 'post', label ="Friction")
plt.step(n_penalty, slip_ps_p,where = 'post', label ="Slip +")
plt.step(n_penalty, slip_ng_p,where = 'post', label ="Slip -")
plt.title("Foot 1 Penalty")
plt.legend()
plt.grid()
plt.axis([1,N,None,None])

plt.subplot(3,2,4)
plt.step(n_force, normal_f,where = 'post', label = 'z')
plt.step(n_force, x_f,where = 'post', label = 'x')
#plt.plot(z,':', color = 'Grey', label = 'z position of the back wheel')
plt.title("Foot 1 Forces")
plt.legend()
plt.grid()
plt.axis([1,N,None,None]);

# foot 2
normal_p = list(m.ground_penalties[:, 'foot2', 'contact'].value)
friction_p = list(m.ground_penalties[:, 'foot2', 'friction'].value)
slip_ps_p = list(m.ground_penalties[:, 'foot2', 'slip_ps'].value)
slip_ng_p = list(m.ground_penalties[:, 'foot2', 'slip_ng'].value)

normal_f = list(m.GRF[:, 'foot2', 'z', 'ng',:].value)
x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        x_f.append(m.GRF[n, 'foot2', 'x', 'ps',p].value - m.GRF[n, 'foot2', 'x', 'ng',p].value) 


plt.subplot(3,2,5)
plt.step(n_penalty,normal_p,where= 'post', label ="Contact")
plt.step(n_penalty,friction_p,where= 'post', label ="Friction")
plt.step(n_penalty,slip_ps_p,where= 'post', label ="Slip +")
plt.step(n_penalty,slip_ng_p,where= 'post', label ="Slip -")
plt.title("Foot 2 Penalty")
plt.legend()
plt.grid()

plt.axis([1,N,None,None])
plt.subplot(3,2,6)
plt.step(n_force,normal_f,where= 'post', label = 'z')
plt.step(n_force,x_f,where= 'post', label = 'x')
#plt.plot(z,':', color = 'Grey', label = 'z position of the back wheel')
plt.title("Foot 2 Forces")
plt.legend()
plt.grid()
plt.axis([1,N,None,None]);

if save == 'y': # yes I know what this looks like but it is there for a reason.
    name = input("Name for Landing: ")
    try:
        path = "../../Results/NEW/2pt/"+name
        os.mkdir(path)
    except FileExistsError:
        print("File Already exitst")
        save = input("Write over data? [y/n]")
    if save == 'y':
        plt.savefig("../../Results/NEW/2pt/"+name+"/Penaltys_and_Forces.pdf")
    
# plot control torque
τ1 = list(m.Tc[:,2].value)
τ2 = list(m.Tc[:,3].value)
τ3 = list(m.Tc[:,4].value)
τ4 = list(m.Tc[:,5].value)

for i in range(len(τ1)):
    τ1[i] *= BW
    τ2[i] *= BW
    τ3[i] *= BW
    τ4[i] *= BW

n = np.linspace(1,N,N)
plt.figure(figsize = (15,5))
plt.step(n,np.array(τ1),where='post', label = "$τ_1$")
plt.step(n,np.array(τ2),where='post', label = "$τ_2$")
plt.step(n,np.array(τ3),where='post', label = "$τ_3$")
plt.step(n,np.array(τ4),where='post', label = "$τ_4$")
plt.grid()
plt.legend()
plt.xlabel("n")
plt.ylabel("Actuator Troque [Nm]")
plt.axis([1,N,None, None])

if save == 'y': # yes I know what this looks like but it is there for a reason.
    #figname = "../../Figures/Leg_"+name+"_Actuator_T.pdf"
    plt.savefig("../../Results/NEW/2pt/"+name+"/Actuator_T.pdf")
    print(save_data_radau(m,N,m.hm,0.02,"../../Results/NEW/2pt/"+name+"/Data"))

In [ ]:
#save = 'n'

fig1 = plt.figure(figsize = (13,7),constrained_layout=True)
gs = fig1.add_gridspec(7, 10)

ax1 = fig1.add_subplot(gs[:, :6]); ax2 = fig1.add_subplot(gs[:, 6:])
ax1.set_aspect('equal');ax2.set_aspect('equal')

update = lambda i: plot_UAV_legs(i,m,ax1,ax2,-1,45,1,-32) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1, N+1),interval = 100,repeat=False)

try:
    if save == "y":
        #name = input("Name for Video: ")
        print("Making video ...")
        Writer = ani.writers['ffmpeg']
        writer = Writer(fps=25, bitrate=1800)
        animate.save("../../Results/NEW/2pt/"+name+"/"+name+".mp4", writer=writer)
except:
    print("")

HTML(animate.to_html5_video())

In [ ]:
# # low drop test that turned in to the stationary test and the hop test.
# for n in range(1,N+1):
#     for l in range(1,4):
#         for dof in DOFs:
#             m.X[n,l,dof].value = 0.0
#             m.dX[n,l,dof].value = 0.0
#             m.ddX[n,l,dof].value = 0.0
#         #m.ddX[n,l,'z'] = 9.81
#     # set all controll stuff to zero
#         m.Tc[n,l].value = 0.0
#         #m.Tc[n,l].fixed = True
    
#     m.T[n].value = 0.0
#     #m.T[n].fixed = True
    
#     m.de[n].value = 0.0
#     #m.de[n].fixed = True
    
#     m.df[n].value = 0.0
#     #m.df[n].fixed = True
    
#     m.V[n] = 0.0
    
#     for contact in contacts:
#         for gc in ground_constraints:
#             m.ground_penalties[n,contact, gc].value = 0.0
        
    

# # initual conditions
# m.X[1,1,'x'].value = 0
# m.X[1,1,'z'].value = -0.5
# m.X[1,1,'th'].value = 0
# m.X[1,1,'x'].fixed = True
# m.X[1,1,'z'].fixed = True
# m.X[1,1,'th'].fixed = True

# m.dX[1,1,'x'].value = 0
# m.dX[1,1,'z'].value = 0
# m.dX[1,1,'th'].value = 0
# m.dX[1,1,'x'].fixed = True
# m.dX[1,1,'z'].fixed = True
# m.dX[1,1,'th'].fixed = True

# m.ddX[1,1,'x'].value = 0
# m.ddX[1,1,'z'].value = 0
# m.ddX[1,1,'th'].value = 0
# #m.ddX[1,1,'x'].fixed = True
# #m.ddX[1,1,'z'].fixed = True
# #m.ddX[1,1,'th'].fixed = True

# m.X[1,2,'x'].value = 0
# m.X[1,2,'z'].value = -0.25 - 0.25/2
# m.X[1,2,'th'].value = 0
# m.X[1,2,'x'].fixed = True
# m.X[1,2,'z'].fixed = True
# m.X[1,2,'th'].fixed = True

# m.dX[1,2,'x'].value = 0
# m.dX[1,2,'z'].value = 0
# m.dX[1,2,'th'].value = 0
# #m.dX[1,2,'x'].fixed = True
# #m.dX[1,2,'z'].fixed = True
# m.dX[1,2,'th'].fixed = True

# m.ddX[1,2,'x'].value = 0
# m.ddX[1,2,'z'].value = 0
# m.ddX[1,2,'th'].value = 0
# #m.ddX[1,2,'x'].fixed = True
# #m.ddX[1,2,'z'].fixed = True
# m.ddX[1,2,'th'].fixed = True

# m.X[1,3,'x'].value = 0.0
# m.X[1,3,'z'].value = -0.25/2 #+ 0.25
# m.X[1,3,'th'].value = 0
# m.X[1,3,'x'].fixed = True
# m.X[1,3,'z'].fixed = True
# m.X[1,3,'th'].fixed = True

# m.dX[1,3,'x'].value = 0
# m.dX[1,3,'z'].value = 0
# m.dX[1,3,'th'].value = 0
# #m.dX[1,3,'x'].fixed = True
# #m.dX[1,3,'z'].fixed = True
# #m.dX[1,3,'th'].fixed = True

# m.ddX[1,3,'x'].value = 0
# m.ddX[1,3,'z'].value = 0
# m.ddX[1,3,'th'].value = 0
# #m.ddX[1,3,'x'].fixed = True
# #m.ddX[1,3,'z'].fixed = True
# m.ddX[1,3,'th'].fixed = True

# #m.bottom[1,3,'z'].value = -10 +0
# #m.bottom[1,3,'z'].fixed = True

# #m.top[1,3,'z'].value = -0.5 +0.5
# #m.top[1,3,'z'].fixed = True

# m.GRF[N,'foot','z','ng'].value = 0 #0.929774855930616
# m.GRF[N,'wheel','z','ng'].value = 0 # 1-0.929774855930616

# #m.top[1,3,'x'].value = -0.25
# #m.bottom[1,2,'x'].value = -0.25


# # final conditions
# m.X[N,1,'z'].value = -1
# m.X[N,1,'z'].fixed = True

In [ ]:
def init_from_solution_for_radau(m, solution_path, legs, P):
    '''
        Initializes pyomo model from a previous solution.
        
        Inputs:
            - m is the pyomo model
            - solution_path is the path to the pickle file with the info of the prevoius solution.
            - legs refers to the number of legs the model has, it which model, (int)(0,1,or 2)
            - P is the number of collocation points (int)
        '''
    if legs < 2:
        print("Sorry Still need to do this one, nothing done")
        return(m)
    if legs == 2:
        infile = open(solution_path, 'rb')
        data = pkl.load(infile)
        infile.close()
        
    N = data['N']
    if N > len(m.N):
        print("The solution has more nodes thant the current model has.")
        print("Adding values for all nodes of the model")
        N = len(m.N)
    
    if N < len(m.N):
        print("The solution has less nodes thant the current model has.")
        print("Adding values up to node ",N)

        for n in range(1,N+1):
            m.alpha[n,P].value = data['a'][n-1]
            m.de[n].value = data['de'][n-1]
            m.df[n].value = data['df'][n-1]
            m.V[n,P].value = data['V'][n-1]
            m.T[n,P].value = data['T'][n-1]
            
            m.X[n,1,'x',P].value = data['x1'][n-1]
            m.X[n,2,'x',P].value = data['x2'][n-1]
            m.X[n,3,'x',P].value = data['x3'][n-1]
            m.X[n,4,'x',P].value = data['x4'][n-1]
            m.X[n,5,'x',P].value = data['x5'][n-1]
            
            m.X[n,1,'z',P].value = data['z1'][n-1]
            m.X[n,2,'z',P].value = data['z2'][n-1]
            m.X[n,3,'z',P].value = data['z3'][n-1]
            m.X[n,4,'z',P].value = data['z4'][n-1]
            m.X[n,5,'z',P].value = data['z5'][n-1]
            
            m.X[n,1,'th',P].value = data['th1'][n-1]
            m.X[n,2,'th',P].value = data['th2'][n-1]
            m.X[n,3,'th',P].value = data['th3'][n-1]
            m.X[n,4,'th',P].value = data['th4'][n-1]
            m.X[n,5,'th',P].value = data['th5'][n-1]
            #---
            m.dX[n,1,'x',P].value = data['dx1'][n-1]
            m.dX[n,2,'x',P].value = data['dx2'][n-1]
            m.dX[n,3,'x',P].value = data['dx3'][n-1]
            m.dX[n,4,'x',P].value = data['dx4'][n-1]
            m.dX[n,5,'x',P].value = data['dx5'][n-1]
            
            m.dX[n,1,'z',P].value = data['dz1'][n-1]
            m.dX[n,2,'z',P].value = data['dz2'][n-1]
            m.dX[n,3,'z',P].value = data['dz3'][n-1]
            m.dX[n,4,'z',P].value = data['dz4'][n-1]
            m.dX[n,5,'z',P].value = data['dz5'][n-1]
            
            m.dX[n,1,'th',P].value = data['dth1'][n-1]
            m.dX[n,2,'th',P].value = data['dth2'][n-1]
            m.dX[n,3,'th',P].value = data['dth3'][n-1]
            m.dX[n,4,'th',P].value = data['dth4'][n-1]
            m.dX[n,5,'th',P].value = data['dth5'][n-1]
            #---
            m.ddX[n,1,'x',P].value = data['ddx1'][n-1]
            m.ddX[n,2,'x',P].value = data['ddx2'][n-1]
            m.ddX[n,3,'x',P].value = data['ddx3'][n-1]
            m.ddX[n,4,'x',P].value = data['ddx4'][n-1]
            m.ddX[n,5,'x',P].value = data['ddx5'][n-1]
            
            m.ddX[n,1,'z',P].value = data['ddz1'][n-1]
            m.ddX[n,2,'z',P].value = data['ddz2'][n-1]
            m.ddX[n,3,'z',P].value = data['ddz3'][n-1]
            m.ddX[n,4,'z',P].value = data['ddz4'][n-1]
            m.ddX[n,5,'z',P].value = data['ddz5'][n-1]
            
            m.ddX[n,1,'th',P].value = data['ddth1'][n-1]
            m.ddX[n,2,'th',P].value = data['ddth2'][n-1]
            m.ddX[n,3,'th',P].value = data['ddth3'][n-1]
            m.ddX[n,4,'th',P].value = data['ddth4'][n-1]
            m.ddX[n,5,'th',P].value = data['ddth5'][n-1]
            #---
            m.Tc[n,2].value = data['tau1'][n-1]
            m.Tc[n,3].value = data['tau2'][n-1]
            m.Tc[n,4].value = data['tau3'][n-1]
            m.Tc[n,5].value = data['tau4'][n-1]
            
            
            for gc in ground_constraints:
                m.ground_penalties[n,'wheel',gc].value = data['wheel penaltys'][gc][n-1]
                m.ground_penalties[n,'foot1',gc].value = data['foot1 penaltys'][gc][n-1]
                m.ground_penalties[n,'foot2',gc].value = data['foot2 penaltys'][gc][n-1]
        
            m.GRF[n,'wheel','z','ng',P].value = data['wheel forces']['normal'][n-1]
            m.GRF[n,'wheel','x','ng',P].value = data['wheel forces']['x_ng'][n-1]
            m.GRF[n,'wheel','x','ps',P].value = data['wheel forces']['x_ps'][n-1]
            
            m.GRF[n,'foot1','z','ng',P].value = data['foot1 forces']['normal'][n-1]
            m.GRF[n,'foot1','x','ng',P].value = data['foot1 forces']['x_ng'][n-1]
            m.GRF[n,'foot1','x','ps',P].value = data['foot1 forces']['x_ps'][n-1]
            
            m.GRF[n,'foot2','z','ng',P].value = data['foot2 forces']['normal'][n-1]
            m.GRF[n,'foot2','x','ng',P].value = data['foot2 forces']['x_ng'][n-1]
            m.GRF[n,'foot2','x','ps',P].value = data['foot2 forces']['x_ps'][n-1]

    print("Values updated")
    return(m)

In [ ]:
m.X.pprint()

In [ ]:
# # high drop test
# m.calc_a.deactivate()

# for n in range(1,N+1):
#     for p in range(1,P+1):
#         for l in range(1,L+1):
#             for dof in DOFs:
#                 m.X[n,l,dof,p].value = 0.0
#                 m.dX[n,l,dof,p].value = 0.0
#                 m.ddX[n,l,dof,p].value = 0.0

#                 for contact in contacts:
#                     for sgn in signs:
#                         m.GRF[n, contact, dof, sgn,p].value = 0.0
#                     for gc in ground_constraints:
#                         m.ground_penalties[n,contact,gc].value = 0.0


#         m.alpha[n,p].fix(0)
#         m.V[n,p].value = 0

#         m.T[n].value = 0.0
#         m.de[n].value = 0.0
#         m.df[n].value = 0.0
#         m.Tc[n,2].value = 0.0
#         m.Tc[n,3].value = 0.0
#         m.Tc[n,4].value = 0.0
#         m.Tc[n,5].value = 0.0

#         m.T[n].fixed = True
#         m.de[n].fixed = True
#         m.df[n].fixed = True
#         m.Tc[n,2].fixed = True
#         m.Tc[n,3].fixed = True
#         m.Tc[n,4].fixed = True
#         m.Tc[n,5].fixed = True
            
        
# # position -----------------------------
# m.X[1,1,'x',P].value = 0.0
# m.X[1,2,'x',P].value = 0.0
# m.X[1,3,'x',P].value = 0.0
# m.X[1,4,'x',P].value = 0.0
# m.X[1,5,'x',P].value = 0.0

# m.X[1,1,'x',P].fixed = True 
# m.X[1,2,'x',P].fixed = True 
# m.X[1,3,'x',P].fixed = True
# m.X[1,4,'x',P].fixed = True 
# m.X[1,5,'x',P].fixed = True 

# m.X[1,1,'z',P].value = -0.5
# m.X[1,2,'z',P].value = m.X[1,1,'z',P].value + m.len[2]/2
# m.X[1,3,'z',P].value = m.X[1,1,'z',P].value + m.len[2] + m.len[3]/2
# m.X[1,4,'z',P].value = m.X[1,1,'z',P].value + m.len[2]/2
# m.X[1,5,'z',P].value = m.X[1,1,'z',P].value + m.len[2] + m.len[3]/2

# m.X[1,1,'z',P].fixed = True 
# m.X[1,2,'z',P].fixed = True 
# m.X[1,3,'z',P].fixed = True
# m.X[1,4,'z',P].fixed = True 
# m.X[1,5,'z',P].fixed = True 

# m.X[1,1,'th',P].value = 0.0 
# m.X[1,2,'th',P].value = 0.0 
# m.X[1,3,'th',P].value = 0.0
# m.X[1,4,'th',P].value = 0.0 
# m.X[1,5,'th',P].value = 0.0

# m.X[1,1,'th',P].fixed = True 
# m.X[1,2,'th',P].fixed = True 
# m.X[1,3,'th',P].fixed = True
# m.X[1,4,'th',P].fixed = True 
# m.X[1,5,'th',P].fixed = True 

# # velocity -----------------------------
# m.dX[1,1,'x',P].value = 0.0 
# m.dX[1,2,'x',P].value = 0.0 
# m.dX[1,3,'x',P].value = 0.0
# m.dX[1,4,'x',P].value = 0.0 
# m.dX[1,5,'x',P].value = 0.0

# m.dX[1,1,'x',P].fixed = True 
# m.dX[1,2,'x',P].fixed = True 
# m.dX[1,3,'x',P].fixed = True
# m.dX[1,4,'x',P].fixed = True 
# m.dX[1,5,'x',P].fixed = True 

# m.dX[1,1,'z',P].value = 0.0 
# m.dX[1,2,'z',P].value = 0.0
# m.dX[1,3,'z',P].value = 0.0
# m.dX[1,4,'z',P].value = 0.0
# m.dX[1,5,'z',P].value = 0.0

# m.dX[1,1,'z',P].fixed = False 
# m.dX[1,2,'z',P].fixed = False 
# m.dX[1,3,'z',P].fixed = False
# m.dX[1,4,'z',P].fixed = False 
# m.dX[1,5,'z',P].fixed = False 

# m.dX[1,1,'th',P].value = 0.0 
# m.dX[1,2,'th',P].value = 0.0 
# m.dX[1,3,'th',P].value = 0.0
# m.dX[1,4,'th',P].value = 0.0 
# m.dX[1,5,'th',P].value = 0.0

# m.dX[1,1,'th',P].fixed = True 
# m.dX[1,2,'th',P].fixed = True 
# m.dX[1,3,'th',P].fixed = True
# m.dX[1,4,'th',P].fixed = True 
# m.dX[1,5,'th',P].fixed = True 

# # acceleration -------------------------
# m.ddX[1,1,'x',P].value = 0.0 
# m.ddX[1,2,'x',P].value = 0.0 
# m.ddX[1,3,'x',P].value = 0.0 
# m.ddX[1,4,'x',P].value = 0.0 
# m.ddX[1,5,'x',P].value = 0.0

# m.ddX[1,1,'x',P].fixed = True 
# m.ddX[1,2,'x',P].fixed = True 
# m.ddX[1,3,'x',P].fixed = True
# m.ddX[1,4,'x',P].fixed = True 
# m.ddX[1,5,'x',P].fixed = True 

# m.ddX[1,1,'z',P].value = 9.81 
# m.ddX[1,2,'z',P].value = 9.81
# m.ddX[1,3,'z',P].value = 9.81
# m.ddX[1,4,'z',P].value = 9.81
# m.ddX[1,5,'z',P].value = 9.81

# m.ddX[1,1,'z',P].fixed = True 
# m.ddX[1,2,'z',P].fixed = True 
# m.ddX[1,3,'z',P].fixed = True 
# m.ddX[1,4,'z',P].fixed = True 
# m.ddX[1,5,'z',P].fixed = True 

# m.ddX[1,1,'th',P].value = 0.0 
# m.ddX[1,2,'th',P].value = 0.0 
# m.ddX[1,3,'th',P].value = 0.0
# m.ddX[1,4,'th',P].value = 0.0 
# m.ddX[1,5,'th',P].value = 0.0

# m.ddX[1,1,'th',P].fixed = True 
# m.ddX[1,2,'th',P].fixed = True 
# m.ddX[1,3,'th',P].fixed = True
# m.ddX[1,4,'th',P].fixed = True 
# m.ddX[1,5,'th',P].fixed = True

# m.GRF[N,'foot1','z','ng',P].fix(1/2)
# m.GRF[N,'foot2','z','ng',P].fix(1/2)
# m.GRF[N,'wheel','z','ng',P].fix(0.0)

# # m.GRF[N,'foot1','x','ps',P].fix(0.0)
# # m.GRF[N,'foot2','x','ps',P].fix(0.0)
# # m.GRF[N,'wheel','x','ps',P].fix(0.0)

# # m.GRF[N,'foot1','x','ps',P].fix(0.0)
# # m.GRF[N,'foot2','x','ps',P].fix(0.0)
# # m.GRF[N,'wheel','x','ps',P].fix(0.0)

In [ ]:
m.ddX.pprint()

In [ ]:
max(list(m.ground_penalties[:,:,:].value))

In [ ]:
# foot 2
normal_p = list(m.ground_penalties[:, 'foot2', 'contact'].value)
friction_p = list(m.ground_penalties[:, 'foot2', 'friction'].value)
slip_ps_p = list(m.ground_penalties[:, 'foot2', 'slip_ps'].value)
slip_ng_p = list(m.ground_penalties[:, 'foot2', 'slip_ng'].value)

normal_f = list(m.GRF[:, 'foot2', 'z', 'ng',:].value)
x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        x_f.append(m.GRF[n, 'foot2', 'x', 'ps',p].value - m.GRF[n, 'foot2', 'x', 'ng',p].value) 

n_force = np.linspace(1,N,N*P)
n_penalty = np.linspace(1,N,N)

plt.figure(figsize = (15,10))
plt.subplot(2,1,1)
plt.step(n_penalty,normal_p,where = 'post', label ="contact")
plt.step(n_penalty,friction_p,where = 'post', label ="Friction")
plt.step(n_penalty,slip_ps_p,where = 'post', label ="Slip +")
plt.step(n_penalty,slip_ng_p,where = 'post', label ="Slip -")
plt.title("Second Foot Penalty")
plt.legend()
plt.grid()
plt.axis([0,N,None,None])
plt.subplot(2,1,2)
plt.step(n_force, normal_f,where= 'post', label = 'z')
plt.step(n_force,x_f,where= 'post', label = 'x')
#plt.plot(z,':', color = 'Grey', label = 'z position of the back wheel')
plt.title("Second Foot Forces")
plt.legend()
plt.grid()
plt.axis([80,N,None,None]);

In [ ]:
def save_data_radau(m,N,hm,h_var,name, legs = True):

    if legs:
        data = {
            'P' : len(m.P),
            'Cost': float(m.Cost.expr()),
            'Cost_type':"Min Penalty",
            'l1':m.len[1],
            'l2':m.len[2],
            'l3':m.len[3],
            'm1':m.m[1],
            'm2':m.m[2],
            'm3':m.m[3],
            'x1': list(m.X[:,1,'x',:].value),
            'z1': list(m.X[:,1,'z',:].value),
            'th1': list(m.X[:,1,'th',:].value),
            'dx1': list(m.dX[:,1,'x',:].value),
            'dz1': list(m.dX[:,1,'z',:].value),
            'dth1': list(m.dX[:,1,'th',:].value),
            'ddx1': list(m.ddX[:,1,'x',:].value),
            'ddz1': list(m.ddX[:,1,'z',:].value),
            'ddth1': list(m.ddX[:,1,'th',:].value),
            'x2': list(m.X[:,2,'x',:].value),
            'z2': list(m.X[:,2,'z',:].value),
            'th2': list(m.X[:,2,'th',:].value),
            'dx2': list(m.dX[:,2,'x',:].value),
            'dz2': list(m.dX[:,2,'z',:].value),
            'dth2': list(m.dX[:,2,'th',:].value),
            'ddx2': list(m.ddX[:,2,'x',:].value),
            'ddz2': list(m.ddX[:,2,'z',:].value),
            'ddth2': list(m.ddX[:,2,'th',:].value),
            'x3': list(m.X[:,3,'x',:].value),
            'z3': list(m.X[:,3,'z',:].value),
            'th3': list(m.X[:,3,'th',:].value),
            'dx3': list(m.dX[:,3,'x',:].value),
            'dz3': list(m.dX[:,3,'z',:].value),
            'dth3': list(m.dX[:,3,'th',:].value),
            'ddx3': list(m.ddX[:,3,'x',:].value),
            'ddz3': list(m.ddX[:,3,'z',:].value),
            'ddth3': list(m.ddX[:,3,'th',:].value),
            'x4': list(m.X[:,4,'x',:].value),
            'z4': list(m.X[:,4,'z',:].value),
            'th4': list(m.X[:,4,'th',:].value),
            'dx4': list(m.dX[:,4,'x',:].value),
            'dz4': list(m.dX[:,4,'z',:].value),
            'dth4': list(m.dX[:,4,'th',:].value),
            'ddx4': list(m.ddX[:,4,'x',:].value),
            'ddz4': list(m.ddX[:,4,'z',:].value),
            'ddth4': list(m.ddX[:,4,'th',:].value),
            'x5': list(m.X[:,5,'x',:].value),
            'z5': list(m.X[:,5,'z',:].value),
            'th5': list(m.X[:,5,'th',:].value),
            'dx5': list(m.dX[:,5,'x',:].value),
            'dz5': list(m.dX[:,5,'z',:].value),
            'dth5': list(m.dX[:,5,'th',:].value),
            'ddx5': list(m.ddX[:,5,'x',:].value),
            'ddz5': list(m.ddX[:,5,'z',:].value),
            'ddth5': list(m.ddX[:,5,'th',:].value),
            'V': list(m.V[:,:].value),
            'a': list(m.alpha[:,:].value),
            'T': list(m.T[:].value),
            'de': list(m.de[:].value),
            'df': list(m.df[:].value),
            #'th_ub':45*np.pi/180,
            #'th_lb':-45*np.pi/180,
            'tau1':list(m.Tc[:,2].value),
            'tau2':list(m.Tc[:,3].value),
            'tau3':list(m.Tc[:,4].value),
            'tau4':list(m.Tc[:,5].value),
            #'alpha_up':19*np.pi/180,
            #'alpha_lb':-19*np.pi/180,
            'N':N,
            'hm':hm,
            'h_var': h_var,
            'wheel penaltys': {
                'contact': list(m.ground_penalties[:,'wheel','contact'].value),
                'friction': list(m.ground_penalties[:,'wheel','friction'].value),
                'slip_ps': list(m.ground_penalties[:,'wheel','slip_ps'].value),
                'slip_ng': list(m.ground_penalties[:,'wheel','slip_ng'].value)
                },
            'foot1 penaltys': {
                'contact': list(m.ground_penalties[:,'foot1','contact'].value),
                'friction': list(m.ground_penalties[:,'foot1','friction'].value),
                'slip_ps': list(m.ground_penalties[:,'foot1','slip_ps'].value),
                'slip_ng': list(m.ground_penalties[:,'foot1','slip_ng'].value)
                },
            'foot2 penaltys': {
                'contact': list(m.ground_penalties[:,'foot2','contact'].value),
                'friction': list(m.ground_penalties[:,'foot2','friction'].value),
                'slip_ps': list(m.ground_penalties[:,'foot2','slip_ps'].value),
                'slip_ng': list(m.ground_penalties[:,'foot2','slip_ng'].value)
                },
            'wheel forces':{
                'normal': list(m.GRF[:,'wheel','z','ng',:].value),
                'x_ps': list(m.GRF[:,'wheel','x','ps',:].value),
                'x_ng': list(m.GRF[:,'wheel','x','ng',:].value)
                },
            'foot1 forces':{
                'normal': list(m.GRF[:,'foot1','z','ng',:].value),
                'x_ps': list(m.GRF[:,'foot1','x','ps',:].value),
                'x_ng': list(m.GRF[:,'foot1','x','ng',:].value)
                },
            'foot2 forces':{
                'normal': list(m.GRF[:,'foot2','z','ng',:].value),
                'x_ps': list(m.GRF[:,'foot2','x','ps',:].value),
                'x_ng': list(m.GRF[:,'foot2','x','ng',:].value)
                    }
            }
    else:
        data = {
            'P': m.P,
            'Cost': float(m.Cost.expr()),
            'x': list(m.X[:,1].value), 
            'z': list(m.X[:,2].value),
            'th': list(m.X[:,3].value),
            'dx': list(m.dX[:,1].value),
            'dz': list(m.dX[:,2].value),
            'dth': list(m.dX[:,3].value),
            'ddx': list(m.ddX[:,1].value),
            'ddz': list(m.ddX[:,2].value),
            'ddth': list(m.ddX[:,3].value),
            'T': list(m.T[:].value),
            'de': list(m.de[:].value),
            'df': list(m.de[:].value),
            'alpha': list(m.alpha[:].value),
            'V': list(m.V[:].value),
            'N':N,
            'hm':hm,
            'h_var': 0.2,
            'Front penaltys': {
                'normal': list(m.ground_penalty[:,'front','normal'].value),
                'friction': list(m.ground_penalty[:,'front','friction'].value),
                'slip_ps': list(m.ground_penalty[:,'front','slip_ps'].value),
                'slip_ng': list(m.ground_penalty[:,'front','slip_ng'].value)
            },
            'Back penaltys': {
                'normal': list(m.ground_penalty[:,'back','normal'].value),
                'friction': list(m.ground_penalty[:,'back','friction'].value),
                'slip_ps': list(m.ground_penalty[:,'back','slip_ps'].value),
                'slip_ng': list(m.ground_penalty[:,'back','slip_ng'].value)
            },
            'Front forces':{
                'normal': list(m.GRF[:,'front',2,'ng'].value),
                'x_ps': list(m.GRF[:,'front',1,'ps'].value),
                'x_ng': list(m.GRF[:,'front',1,'ng'].value)
            },
            'Back forces':{
                'normal': list(m.GRF[:,'back',2,'ng'].value),
                'x_ps': list(m.GRF[:,'back',1,'ps'].value),
                'x_ng': list(m.GRF[:,'back',1,'ng'].value)
            }
        }
            
    outfile = open(name,'wb')
    pkl.dump(data,outfile)
    outfile.close()
    return("Data Saved")